Basic example of provision calculation when demands in buildings are calculated and the adjacency matrix is computed. Threshold - the value above which buildings will be marked as not meeting the specified value.

In [1]:
import networkx as nx
from objectnat import get_service_provision
import geopandas as gpd
import pandas as pd

builds = gpd.read_parquet("examples_data/buildings.parquet").to_crs(32636)
services = gpd.read_parquet("examples_data/services.parquet").to_crs(32636)
adjacency_matrix = pd.read_parquet('examples_data/matrix_time.parquet')

adjacency_matrix.index = adjacency_matrix.index.astype(int)
services.index = services.index.astype(int)
builds.index = builds.index.astype(int)
adjacency_matrix.columns = adjacency_matrix.columns.astype(int)

build_prov, services_prov, links_prov = get_service_provision(
    buildings=builds,
    services=services,
    adjacency_matrix=adjacency_matrix,
    threshold=10,
)

# Visualize data
m1 = build_prov[build_prov['demand'] > 0].reset_index().explore(column='avg_dist', cmap='Spectral_r',tiles='CartoDB positron')
links_prov[links_prov['distance'] < 10].explore(m=m1,column='service_index', cmap='prism',style_kwds = {'opacity':0.5})
services_prov.explore(m=m1,color='red')

KeyboardInterrupt: 